<a href="https://colab.research.google.com/github/sallumandya1995/meme_world/blob/main/Copy_of_mem_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/spaces/Xhaheen/meme_world
%cd  /content/meme_world

In [ ]:
 # Step 1: Install the necessary packages
!pip install gradio
!pip install openai
!pip install transformers
!pip install Pillow

In [ ]:


# Step 2: Set up the Gradio interface and import necessary packages
import gradio as gr
import openai
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

# Step 3: Load the provided image captioning model
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 4: Create a function to generate captions from images
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def generate_caption(image):
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    if image.mode != "RGB":
        image = image.convert(mode="RGB")
    pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    output_ids = model.generate(pixel_values, **gen_kwargs)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    return caption


# Step 5: Create a function to generate memes using the GPT-3 API
def generate_meme(caption, department):
    openai.api_key = "sk- "
    prompt = f"Create a non-offensive meme caption for the following image description in the context of {department} department: {caption}"
    response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, max_tokens=50, n=1, stop=None, temperature=0.7)
    meme_caption = response.choices[0].text.strip()
    return meme_caption

# Step 6: Define the main meme generation function
def meme_generator(image, department):
    caption = generate_caption(image)
    meme_caption = generate_meme(caption, department)
    return meme_caption

examples = [f"example{i}.png" for i in range(1,7)]

# Step 7: Launch the Gradio application
image_input = gr.inputs.Image()
department_input = gr.inputs.Dropdown(choices=["data science", "product management","marketing","startup" ,"agile","crypto" , "SEO" ])
output_text = gr.outputs.Textbox()

gr.Interface(fn=meme_generator, inputs=[image_input, department_input], outputs=output_text, title="Meme world!",description= " Looking for a fun and easy way to generate memes? Look no further than Meme world! Leveraging large language models like GPT-3PT-3  / Ai21 / Cohere, you can create memes that are sure to be a hit with your friends or network. Created with ♥️ by Arsalan @[Xaheen](https://www.linkedin.com/in/sallu-mandya/). kindly share your thoughts in discussion session and use the app responsibly #NO_Offense \n \n built with ❤️   @[Xhaheen](https://www.linkedin.com/in/sallu-mandya/)",        theme="grass",
    
        examples =[['example5.png','data science' ],
            ['example2.png','product management'],
            ['example3.png','startup'],
            ['example4.png','marketing'],
            ['example1.png','agile'],
            ['example6.png','crypto']]).launch(debug=True)
